## About this Project:

This project tries to create the fundemental metrics and ways of analyzing the texual data.


#### Importing Modules

In [54]:
import pandas as pd
import datetime
from GoogleNews import GoogleNews
from newspaper import Article
import nltk
from newspaper import Config
import time
from newspaper import Article
import string
import numpy as np
import re
import ast
import plotly.express as px
from sentence_transformers import SentenceTransformer
pd.set_option('display.max_rows', None)
from sklearn.cluster import KMeans
import neattext as nt
from textblob import TextBlob
from tqdm.notebook import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/balajisrirajduraisamyushamala/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/balajisrirajduraisamyushamala/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Configurations

In [55]:
#config will allow us to access the specified url for which we are #not authorized. Sometimes we may get 403 client error while parsing #the link to download the article.
nltk.download('punkt')
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 15

############################################
region = 'IN'
start_date =  datetime.date(2023, 1, 24)  #MM/DD/YYYY #datetime.date(2022, 1, 25)'01/24/2023'
end_date =  datetime.date(2023, 3, 31)
no_of_days_from_start_date = end_date - start_date
no_of_days_from_start_date = no_of_days_from_start_date.days
interval_days = 3
periods = round(no_of_days_from_start_date/interval_days)

search_string = 'Adani'
lang='en'
region='IN'
no_of_pages = 3
sleep_time = 20.0

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/balajisrirajduraisamyushamala/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Extract Data

#### Extract Meta Data

This part of the code will skip through the google news and exracts the news article links.

In [ ]:
    # Contains the final results = news filtered by the criteria 
    # (news that in their description contains the search term).
final_results = []
# Initial list of results - it will contain a list of dictionaries (dict).
results = []
for period in range(1,periods+1):
    
        if period == 1:
            dynamic_end_date = start_date+datetime.timedelta(days=interval_days)
            dynamic_start_date = start_date
        else:
            dynamic_start_date = dynamic_end_date+datetime.timedelta(days=1)
            dynamic_end_date = dynamic_start_date+datetime.timedelta(days=interval_days)
        
        print(f'Getting News for the period {dynamic_start_date} to {dynamic_end_date}')
        googlenews=GoogleNews(start=str(dynamic_start_date.strftime("%m/%d/%Y")),end=str(dynamic_end_date.strftime("%m/%d/%Y")),lang=lang, region=region) # I suppose the date is in "MM/dd/yyyy" format...
        googlenews.search(search_string)
        
    
    # Get first 10 pages with the results and append those results to the list - you can set any other range according to  your needs: 
        for page in range(1,no_of_pages+1):
            print(f'        Getting News for the period {dynamic_start_date} to {dynamic_end_date}, Page NO : {page}')
            print('Waiting..')
            time.sleep(sleep_time)
            googlenews.getpage(page) # Consider add an timer for avoid multiple calls and get "HTTP Error 429: Too Many Requests" error.
            results.extend(googlenews.result(sort=True))

In [ ]:
# Remove duplicates and include to the "final_results" list 
# only the news that includes in their description the search term: 
for item in results: 
            #if (item not in final_results and (search_string in item["desc"])):
                final_results.append(item)
# Build and show the final dataframe: 
df=pd.DataFrame(final_results)
df.to_csv(f'./data/raw_data/Meta_data_{search_string}.csv',index=False)

#### Extract Article Data

Now that we have URLs of the article from Meta Data, we can use news3k Article module to extract the full Article text with few other information.

In [61]:
df = pd.read_csv(f'./data/raw_data/Meta_data_{search_string}.csv')
df = df.drop_duplicates()
df.reset_index(drop=True,inplace=True)

In [66]:
sorted(df['media'].unique())

['ABC',
 'ABP LIVE',
 'AP News',
 'Adani Watch',
 'Ahmedabad Mirror',
 'Al Jazeera',
 'Al-Monitor',
 'BBC',
 'BNN Bloomberg',
 'BQ Prime',
 'Bar and Bench',
 'Bloomberg.com',
 'Business Standard',
 'Business Today',
 'CNBC',
 'CNBCTV18.com',
 'CNN',
 'Cartoq',
 'CleanTechnica',
 'Current Affairs',
 'Daily Pakistan',
 'Daily Pioneer',
 'Dredging Today',
 'ET EnergyWorld',
 'ET Infra',
 'ETEnergyworld',
 'ETEnergyworld.com',
 'ETF Stream',
 'Financial Times',
 'Fitch Ratings',
 'Forbes',
 'Foreign Policy',
 'Fortune',
 'Gulf News',
 'Hellenic Shipping News Worldwide',
 'Hindustan Times',
 'ICIS',
 'IFR',
 'India Today',
 'Indian National Congress',
 'Indtoday',
 'Inventiva',
 'Live Law',
 'Middle East Eye',
 'Mint',
 "Moody's",
 'NDTV.com',
 'NPR',
 'Nikkei Asia',
 'Onmanorama',
 'OpIndia',
 'Outlook India',
 'PR News',
 'PSU Watch',
 'PTI',
 'PV Magazine',
 'Quartz',
 'Recharge News',
 'Rediff.com',
 'Republic',
 'Reuters',
 'Scroll',
 'Slate Magazine',
 'SwarajyaMag',
 'TIME',
 'Telang

In [51]:
list = []
failed_list = []
for ind in tqdm(df.index):
    dict={}
    url = (df[df.index ==ind]['link']).to_list()[0]
    article = Article(url)
    try:
        article.download()
        article.parse()
        article.nlp()
        dict['Meta_Publisher'] = (df[df.index ==ind]['media']).to_list()[0]
        dict['Meta_datetime'] = (df[df.index ==ind]['datetime']).to_list()[0]
        dict['Meta_title'] = (df[df.index ==ind]['title']).to_list()[0]
        dict['Meta_link'] = (df[df.index ==ind]['link']).to_list()[0]
        dict['Meta_desc'] = (df[df.index ==ind]['desc']).to_list()[0]
        try:
            dict['Published_Date']=str(article.publish_date)
        except:
            dict['Published_Date']=str(article.publish_date)
        
        try:
            dict['Title']=article.title
        except:
            dict['Title']='NA'
        
        try:
            dict['Article']=article.text
        except:
            dict['Article']='NA'
        
        try:
            dict['Summary']=article.summary
        except:
            dict['Summary']='NA'
        
        try:
            dict['Authors'] = article.authors
        except:
            dict['Authors'] = 'NA'
        
        try:
            dict['Keywords'] = article.keywords
        except:
            dict['Keywords'] = 'NA'   
        
        list.append(dict)
    
    except:
        failed_list.append(ind)
        dict['Meta_Publisher'] = (df[df.index ==ind]['media']).to_list()[0]
        dict['Meta_datetime'] = (df[df.index ==ind]['datetime']).to_list()[0]
        dict['Meta_title'] = (df[df.index ==ind]['title']).to_list()[0]
        dict['Meta_link'] = (df[df.index ==ind]['link']).to_list()[0]
        dict['Meta_desc'] = (df[df.index ==ind]['desc']).to_list()[0]
        dict['Published_Date']=str(article.publish_date)
        dict['Title']='NA'
        dict['Article']='NA'
        dict['Summary']='NA'
        dict['Authors'] = 'NA'
        dict['Keywords'] = 'NA'
        list.append(dict)

  0%|          | 0/741 [00:00<?, ?it/s]

In [52]:
news_df=pd.DataFrame(list)
#news_df.to_csv('./data/processed_data/extracted_data_new.csv',index=False)
print('Number Failed Articles : ',len(failed_list)/len(df.index),'%')

Number Failed Articles :  0.08906882591093117 %


### Process Data

In [3]:
df_raw = pd.read_csv('./data/processed_data/extracted_data_new.csv')

In [4]:
fillna_cols = ['Title','Article','Summary','Authors','Keywords']
df_raw[fillna_cols] = df_raw[fillna_cols].fillna('NA') #Filling NA for the cols where we have null values
#Removing rows where we do not have Article text
df_raw = df_raw[df_raw['Article'] != 'NA']
df_raw['index'] = df_raw.index # Creating an index column
df_raw['Authors_Count'] = df_raw.Authors.transform(ast.literal_eval).str.len() # Getting number of Authors, if there are no Authors info then it would be set to 0.

In [5]:
authors_list = []
#Removing the list format
for authors in df_raw['Authors']:
    temp = authors[1:-1].replace("'", "")
    if temp == '':
        authors_list.append('Not Available')    
    else:
        authors_list.append(temp)
df_raw.Authors = authors_list

In [6]:
# #Replacing dt column from meta where we dont have datetime
# df_raw['Published_Date'] = np.where(df_raw['Published_Date'] == 'None', df_raw['Meta_datetime'], df_raw['Published_Date'])
# #df_raw['Published_Date'] = np.where(df_raw['Published_Date'] == 'None', df_raw['Meta_datetime'], '2023-03-31 00:00:00')
# #df_raw = df_raw[~df_raw['Published_Date'].isna()] # Removing rows where we dont have dt
# df_raw['Published_Date_clean'] = np.where(df_raw['Published_Date'].str.contains('/'), 
#                                           pd.to_datetime(df_raw['Published_Date'],infer_datetime_format=True), 
#                                           pd.to_datetime(df_raw['Published_Date'],infer_datetime_format=True, dayfirst=True)) #.dt.strftime('%m/%d/%Y')
# df_raw['Published_Date_clean'] = pd.to_datetime(df_raw['Published_Date_clean'],utc=True, infer_datetime_format=True).dt.strftime('%d-%m-%Y')
# #df_raw = df_raw[(df_raw['Published_Date_clean']>= '24-02-2023') & (df_raw['Published_Date_clean']<= '31-03-2023')]

In [7]:
# df_temp = df_raw.groupby(['Meta_Publisher'],as_index=False)['Article'].count().sort_values(by = ['Article'],ascending=False)
# df_temp['Meta_Publisher'] = np.where(df_temp['Article'] <=2, 'Others', df_temp['Meta_Publisher'])

# fig = px.pie(df_temp, values='Article', names='Meta_Publisher', title='Number of Articles Published by Media')
# fig.show()

#### Cleaning the text

Remove noise
- stopwords
- special characters
- punctuations
- hashtags
- @userhandles
- extra spaces
- emails
- special chars
- & many more


In [8]:
failed_indexs = []
for col in ['Meta_title','Meta_desc','Title','Article','Summary']:
    for ind in df_raw.index :
        try:
            sentx = nt.TextFrame(text=df_raw[col][ind])
            text = sentx.remove_hashtags()
            text = text.remove_userhandles()
            text = text.remove_bad_quotes()
            text = text.remove_emails()
            text = text.remove_currency_symbols()
            text = text.remove_dates()
            text = text.remove_phone_numbers()
            text = text.remove_multiple_spaces()
            text = text.remove_html_tags()
            text = text.remove_emojis()
            text = text.remove_emails()
            text = text.remove_puncts()
            text = text.remove_urls()
            text = text.remove_special_characters()
            text = text.remove_stopwords()
            text = str(text)
            text = text.replace("\n", "")
            text = text.replace("’s", "")
            df_raw[col][ind] = text
        except:
            failed_indexs.append(ind)
            pass
    print(f'Cleaning Completed for : {col}')
print('Failed Cases : ',len(failed_indexs))

Cleaning Completed for : Meta_title
Cleaning Completed for : Meta_desc
Cleaning Completed for : Title
Cleaning Completed for : Article
Cleaning Completed for : Summary
Failed Cases :  0


In [9]:
#Creating empty columns with default value of 0
df_raw['Article_words_count'] = 0
df_raw['Article_consonants_count'] = 0
df_raw['Article_vowels_count'] = 0
df_raw['Title_words_count'] = 0
df_raw['Meta_title_words_count'] = 0

In [10]:
failed_indexs = []

for ind in df_raw.index :
        try:
            sentx = nt.TextFrame(text=df_raw['Article'][ind])
            df_raw['Article_words_count'][ind]  = sentx.word_stats()['Length of Text']
            df_raw['Article_consonants_count'][ind]  = sentx.word_stats()['Num of Consonants']
            df_raw['Article_vowels_Count'][ind]  = sentx.word_stats()['Num of Vowels']
        except:
            failed_indexs.append(ind)
            pass
for ind in df_raw.index :
        try:
            sentx = nt.TextFrame(text=df_raw['Title'][ind])
            df_raw['Title_words_count'][ind]  = sentx.word_stats()['Length of Text']
        except:
            failed_indexs.append(ind)
            pass
for ind in df_raw.index :
        try:
            sentx = nt.TextFrame(text=df_raw['Meta_title'][ind])
            df_raw['Meta_title_words_count'][ind]  = sentx.word_stats()['Length of Text']
        except:
            failed_indexs.append(ind)
            pass

In [11]:
#Grouping similar media houses to same entity
df_raw.loc[df_raw['Meta_Publisher'].isin(['ET EnergyWorld', 'ET Infra','ETEnergyworld','ETEnergyworld.com',]),'Meta_Publisher'] = 'ET'
df_raw.loc[df_raw['Meta_Publisher'].isin(['The Hindu Business Line']),'Meta_Publisher'] = 'The Hindu'
df_raw.loc[df_raw['Meta_Publisher'].isin(['Yahoo Finance']),'Meta_Publisher'] = 'Yahoo'
df_raw.loc[df_raw['Meta_Publisher'].isin(['CNBCTV18.com']),'Meta_Publisher'] = 'CNBC'

In [21]:
df_temp = pd.read_csv('./data/processed_data/extracted_data_new1.csv')

In [24]:
df_temp['Meta_Publisher'].value_counts()

Business Today                      108
Mint                                 80
The Economic Times                   57
CNBC                                 35
Financial Times                      28
The Hindu                            28
The Wire                             22
Reuters                              21
Times of India                       19
Bloomberg.com                        14
BQ Prime                             13
Hindustan Times                      12
Business Standard                    11
BBC                                  11
ET                                    9
Al Jazeera                            8
CNN                                   8
ThePrint                              7
AP News                               5
Fortune                               5
The Guardian                          4
Nikkei Asia                           3
India Today                           3
Foreign Policy                        3
Yahoo                                 3


### Analysis & Insights

- How many Articles are there?
    - 575

- What are its timeline?
    - 2023-01-24 to 2023-04-02

- How many media houses are there?
    - 71

- How many articles those media houses has produced, Top 5 publishers?
    - Business Today                      108
    - Mint                                 80
    - The Economic Times                   57
    - CNBC                                 35
    - The Hindu                            28
    

- Which Media house has very long Articles?
    - Business Today



In [12]:
print(df_raw['Article'].count())
print(df_raw[df_raw['Published_Date']!= 'None']['Published_Date'].min())
print(df_raw[df_raw['Published_Date']!= 'None']['Published_Date'].max())
print(df_raw['Meta_Publisher'].nunique())
print((df_raw['Meta_Publisher'].value_counts()[0:5]))
print(df_raw[df_raw['Article_words_count'] == df_raw['Article_words_count'].max()]['Meta_Publisher'])

print(df_raw[df_raw['Article_words_count'] == df_raw['Title_words_count'].max()]['Meta_Publisher'])

575
2023-01-24 00:00:00
2023-04-02 00:20:36+05:30
71
Business Today        108
Mint                   80
The Economic Times     57
CNBC                   35
The Hindu              28
Name: Meta_Publisher, dtype: int64
221    Business Today
Name: Meta_Publisher, dtype: object
Series([], Name: Meta_Publisher, dtype: object)


#### Sentiment Analysis using Pre Trained Model - finbert (Financial BERT Model)


The transformer model supports only 512 tokens at a time so the approach is as stated below

High-Level Approach
Getting Started
- Data
- Initialization
Tokenization
Preparing The Chunks
- Split
- CLS and SEP
- Padding
- Reshaping For BERT
Making Predictions

In [13]:
# tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
# model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

In [14]:
df_raw['sentiment_flow']  = np.nan
df_raw['sentiment_flow_score'] = np.nan

In [15]:
def get_sentiment(tokens):
    outputs = model(**tokens)
    probabilities = torch.nn.functional.softmax(outputs[0],dim = -1)

def chunk_text_to_window_size_and_predict_prob(input_ids,attention_mask,total_length,window_length):
    proba_list = []
    start = 0
    window_length = window_length

    loop =True
    while loop:
        end = start + window_length
        if end>= total_length:
            loop = False
            end = total_length
        # Define teh text chunk
        input_id_chunk = input_ids[start:end]
        attention_mask_chunk = attention_mask[start:end]

        # Append [CLS] and [SEP]
        input_id_chunk = [101] + input_id_chunk + [102]
        attention_mask_chunk = [1] + attention_mask_chunk + [1]

        # Convert type list to pytroch tensor
        input_dict = {
            'input_ids' : torch.Tensor([input_id_chunk]).long(),
            'attention_mask' : torch.Tensor([attention_mask_chunk]).int()
        }

        outputs = model(**input_dict)
        probabilities = torch.nn.functional.softmax(outputs[0],dim = -1)
        proba_list.append(probabilities)
        start = end

    return proba_list

def get_mean_from_proba(proba_list,index):
    with torch.no_grad():
        ###
        temp_sentiment = []
        temp_sentiment_score = []
        for list in proba_list:
            with torch.no_grad():
                temp_score = round(list.max().item(),2)
                temp_sentiment_score.append(temp_score)
                temp_senti = torch.argmax(list).item()
                temp_sentiment.append(temp_senti)
            df_raw['sentiment_flow'] [index] = str(temp_sentiment)
            df_raw['sentiment_flow_score'][index] = str(temp_sentiment_score)
        ###
        stacks = torch.stack(proba_list)
        stacks = stacks.resize(stacks.shape[0],stacks.shape[2])
        mean = stacks.mean(dim = 0)
        score = round(mean.max().item(),2)
        mean = torch.argmax(mean).item()
    return mean ,score

In [ ]:
Article_Overall_Sentiment = []
Article_Overall_Sentiment_score = []
for index in tqdm(df_raw['index']):
    tokens = tokenizer.encode_plus( df_raw['Article'][index],add_special_tokens=False)
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    proba_list = chunk_text_to_window_size_and_predict_prob(input_ids,attention_mask,total_length = len(input_ids),window_length = int(len(input_ids)/11))
    article_sentiment,score = get_mean_from_proba(proba_list,index)
    Article_Overall_Sentiment.append(article_sentiment)
    Article_Overall_Sentiment_score.append(score)

In [ ]:
df_raw['Article_Overall_Sentiment'] = Article_Overall_Sentiment
df_raw['Article_Overall_Sentiment_score'] = Article_Overall_Sentiment_score
replace_values = {0: "negative",
                    1: "neutral",
                    2: "positive"}
df_raw['Article_Overall_Sentiment'] = df_raw['Article_Overall_Sentiment'].map(replace_values)
df_raw = df_raw.sort_values(['Meta_datetime','Meta_Publisher'],ascending=True).reset_index(drop=True)
df_raw['index'] = df_raw.index
df_raw.to_csv('./data/processed_data/extracted_data_new1.csv',index=False)

In [16]:
#Reading Processed data
df_raw = pd.read_csv('./data/processed_data/extracted_data_new1.csv')

#### Emotion of the Article

In [17]:
# tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
# model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
from transformers import AutoTokenizer, AutoModelWithLMHead
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")

In [18]:
df_raw['emotion_flow']  = np.nan
df_raw['emotion_flow_score'] = np.nan

In [19]:
def chunk_text_to_window_size_and_predict_prob(input_ids,total_length,window_length):
    proba_list = []
    start = 0
    input_ids = input_ids[0]
    window_length = window_length
    loop =True
    while loop:
        end = start + window_length
        if end>= total_length:
            loop = False
            end = total_length
        # Define teh text chunk
        input_id_chunk = input_ids[start:end]
        outputs = model.generate(input_ids=input_id_chunk.unsqueeze(0),
               max_length=2)
        dec = [tokenizer.decode(ids) for ids in outputs]
        label = dec[0]
        proba_list.append(label)
        start = end
    return proba_list

In [20]:
Article_Overall_Emotion = []
Article_Overall_Emotion_flow = []
for index in tqdm(df_raw['index']):
        input_ids = tokenizer.encode(df_raw['Article'][index] + '</s>', return_tensors='pt')
        proba_list = chunk_text_to_window_size_and_predict_prob(input_ids,total_length = len(input_ids[0]),window_length = int(len(input_ids[0])/11))
        #getting max emotion
        overall_emotion = Counter(proba_list)
        overall_emotion = Counter(proba_list)
        overall_emotion = overall_emotion.most_common(1)
        Article_Overall_Emotion.append(overall_emotion)
        Article_Overall_Emotion_flow.append(proba_list)     

  0%|          | 0/575 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (932 > 512). Running this sequence through the model will result in indexing errors


In [25]:
df_raw['Article_Overall_Emotion'] = Article_Overall_Emotion
df_raw['Article_Overall_Emotion_flow'] = Article_Overall_Emotion_flow

In [26]:
df_raw.to_csv('temp.csv',index = False)

In [27]:
# Creating Features
df_raw[['Article_polarity', 'Article_subjectivity']] = df_raw['Article'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
df_raw[['Title_polarity', 'Tile_subjectivity']] = df_raw['Title'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
df_raw[['Meta_Title_polarity', 'Meta_Tile_subjectivity']] = df_raw['Meta_title'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))


What is the correlation between Subjectivity & Negative/Positive Sentiment Score?

In [28]:
df_temp = df_raw.groupby(['Meta_Publisher','Article_Overall_Sentiment'],as_index = False).agg({'Article_subjectivity':'mean',
                                                                                            'Article_polarity':'mean',
                                                                                            "Article_Overall_Sentiment_score":'mean'})

In [29]:
df_temp['Article_Overall_Sentiment_score'] = df_temp['Article_Overall_Sentiment_score'].abs()

In [30]:
fig = px.scatter(df_temp, x="Article_subjectivity", y="Article_polarity", color="Article_Overall_Sentiment",
                                                trendline = 'lowess',#ols
                                                size="Article_Overall_Sentiment_score")

fig.update_layout(title =
       'Relation between Article Subjectivity and Article Polarity',
        xaxis_title = 'Article_subjectivity', yaxis_title = 'Article_polarity',
        width = 1000, height = 700)
fig.show()

#### Insights 
 - From the above chart we can see there is moderate relationship between personal opinions of the authour & overall polarity of the article.
 - There is >50% of contribution for the Article's negative overall sentiment due to subjectivity of the Author.
 - When the Subjectivity increases the overall positive sentiment of the article decreases, this show personal opinions will lead to over all sentiment of the article.
 - Author's high subjectivity lean towards overall Negative sentiments with respect to Adani enterprises.

In [31]:
df_raw.loc[df_raw.Article_Overall_Sentiment == "positive", "Article_Overall_Sentiment_score"] = df_raw.loc[df_raw.Article_Overall_Sentiment == "positive"]['Article_Overall_Sentiment_score']+1
df_raw.loc[df_raw.Article_Overall_Sentiment == "negative", "Article_Overall_Sentiment_score"] = df_raw.loc[df_raw.Article_Overall_Sentiment == "negative"]['Article_Overall_Sentiment_score']-1

In [32]:
fig = px.line(df_raw, x="index", y="Article_Overall_Sentiment_score",line_shape='spline')
fig.show()

How to understand the article?

In [33]:
index_value = 0 # Input index here
req_cols = ['Meta_Publisher','Title','sentiment_flow','sentiment_flow_score','Article_Overall_Sentiment','Article_Overall_Sentiment_score']
df_temp = df_raw[df_raw['index'] == index_value]

print('Media:',df_temp['Meta_Publisher'][0])
print('Title:',df_temp['Title'][0])
print('Over all Sentiment of the Article is',df_temp['Article_Overall_Sentiment'][0])
print('Over all Sentiment Score of the Article is',df_temp['Article_Overall_Sentiment_score'][0])
print('Sentiment Flow of the Article is',df_temp['sentiment_flow'][0])
print('The over all way the article is written is it starts negatively then ending at a Positive note')

Media: AP News
Title: Indian billionaire Adani center 68B stock market rout
Over all Sentiment of the Article is negative
Over all Sentiment Score of the Article is -0.52
Sentiment Flow of the Article is [0, 0, 0, 2, 1, 2, 0, 1, 1, 0, 0, 1]
The over all way the article is written is it starts negatively then ending at a Positive note


In [34]:
print(df_raw['Meta_Publisher'].unique())
lis_of_media = ['The New York Times', 'The Economic Times', 'Mint']
df_temp = df_raw[df_raw['Meta_Publisher'].isin(lis_of_media)]

fig = px.scatter_3d(df_temp, x='Article_subjectivity', y='Meta_Publisher', z='Title_polarity',
              color='Article_Overall_Sentiment')
fig.show()

['AP News' 'Bloomberg.com' 'Gulf News' 'Mint' 'Reuters'
 'The Economic Times' 'The Hindu' 'Business Today' 'Daily Pakistan'
 'Yahoo' 'The New York Times' 'Zee Business' 'Business Standard' 'BBC'
 'ET' 'Inventiva' 'The Washington Post' 'Times of India' 'Al Jazeera'
 'CNBC' 'The Guardian' 'Financial Times' 'BQ Prime' 'CNN'
 'Hindustan Times' 'Live Law' 'Fortune' 'fDi Intelligence' 'The Telegraph'
 'India Today' 'ETF Stream' 'TIME' 'The Wire' 'Nikkei Asia' 'WSWS'
 'Foreign Policy' 'Recharge News' 'The Diplomat' 'PSU Watch'
 'The Daily Star' 'The Morning Context' 'BNN Bloomberg' 'Outlook India'
 'PV Magazine' 'CleanTechnica' 'Adani Watch' 'Rediff.com'
 'The Canberra Times' 'bdnews24.com' 'Middle East Eye' 'ThePrint'
 'NDTV.com' 'The Moodie Davitt Report' 'Cartoq' 'Daily Pioneer'
 'Telangana Today' 'Telegraph India' 'TotalEnergies.com' 'ICIS'
 'Dredging Today' 'Tribune India' 'Hellenic Shipping News Worldwide'
 'OpIndia' 'Ahmedabad Mirror' 'The Financial Express' 'Onmanorama'
 'Indtoday' 'B

In [35]:
#Coorelation Plot

fig = px.scatter(df_raw, x="Article_subjectivity", y="Title_polarity", color="Article_Overall_Sentiment",
                 size='Article_Overall_Sentiment_score', hover_data=['Article_Overall_Sentiment','Meta_Publisher'])
print('''Most of the Article Titles are neutral in nature, and the articles that has positive sentiments 
        tends to have more personal opinion than negative sentiment articles. 
        
        The Media house with least subjectivity is BNN Bloomberg &
        Most Subjectivity is Bloomberg.com followed by CNBC,BBC,The Hindu, Business Standard.
        Mint has the most positive and high subjectivity and Economic Times has the least 
        subjectivity with overll positive sentiments.

        Clean Technica has the high subjectivity and negative sentiments followed by TOI
        Business Today has the high Polarity with Positive sentiment 
        ''')
fig.show()


ValueError: 
    Invalid element(s) received for the 'size' property of scatter.marker
        Invalid elements include: [-0.52, -0.5900000000000001, -0.54, -0.45999999999999996, -0.5700000000000001, -0.6, -0.43000000000000005, -0.5, -0.27, -0.51]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above

In [ ]:
df_temp = pd.DataFrame()
df_temp['sentiment_flow_score'] = pd.DataFrame(ast.literal_eval(df_raw['sentiment_flow_score'][0]))
df_temp['sentiment_flow'] = pd.DataFrame(ast.literal_eval(df_raw['sentiment_flow'][23]))
fig = px.scatter(x=df_temp.index, y=df_temp.sentiment_flow_score,color=df_temp.sentiment_flow, title='Flow of Sentiment')
fig.show()


In [36]:
#Coorelation Plot

fig = px.scatter(df_raw, x="Article_Overall_Sentiment_score", y="Article_subjectivity", color="Article_Overall_Sentiment",
                 size='Article_Overall_Sentiment_score', hover_data=['Article_Overall_Sentiment'])
fig.show()


ValueError: 
    Invalid element(s) received for the 'size' property of scatter.marker
        Invalid elements include: [-0.52, -0.5900000000000001, -0.54, -0.45999999999999996, -0.5700000000000001, -0.6, -0.43000000000000005, -0.5, -0.27, -0.51]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above

In [ ]:
#Coorelation Plot

fig = px.scatter(df_raw, x="Article_subjectivity", y="Article_polarity", color="Meta_Publisher",
                 size='Article_Overall_Sentiment_score', hover_data=['Article_Overall_Sentiment'])
fig.show()


In [ ]:
df_raw.columns

#### Text Processing & Feature generation

Polarity is the output that lies between [-1,1], where -1 refers to negative sentiment and +1 refers to positive sentiment. Subjectivity is the output that lies within [0,1] and refers to personal opinions and judgments.

In [ ]:
df_temp = df_raw.groupby(['Meta_Publisher'],as_index=False).agg({'Article_polarity':'mean', 'Article_subjectivity':'mean','Title_polarity':'mean','Tile_subjectivity':'mean'})
fig = px.scatter(df_temp, x="Article_polarity", y="Article_subjectivity", color="Meta_Publisher",
                  hover_data=['Article_polarity','Article_subjectivity','Title_polarity','Tile_subjectivity'])

fig.show()
fig = px.density_heatmap(df_temp, x="Article_polarity", y="Article_subjectivity",text_auto=True,hover_data=['Meta_Publisher'])
fig.show()

In [ ]:
lemma = WordNetLemmatizer()
stop_words = stopwords.words('english')
def text_prep(x):
     corp = str(x).lower() 
     corp = re.sub('[^a-zA-Z]+',' ', corp).strip() 
     tokens = word_tokenize(corp)
     words = [t for t in tokens if t not in stop_words]
     lemmatize = [lemma.lemmatize(w) for w in words]
    
     return lemmatize

In [ ]:
preprocess_tag = [text_prep(i) for i in df_raw['Article']]
df_raw["Article_preprocess_txt"] = preprocess_tag

df_raw['Article_total_len'] = df_raw['Article_preprocess_txt'].map(lambda x: len(x))
file = open('negative-words.txt', 'r')
neg_words = file.read().split()
file = open('positive-words.txt', 'r')
pos_words = file.read().split()
num_pos = df_raw['Article_preprocess_txt'].map(lambda x: len([i for i in x if i in pos_words]))
df_raw['Article_pos_count'] = num_pos
num_neg = df_raw['Article_preprocess_txt'].map(lambda x: len([i for i in x if i in neg_words]))
df_raw['Article_neg_count'] = num_neg
df_raw['Article_sentiment'] = round((df_raw['Article_pos_count'] - df_raw['Article_neg_count']) / df_raw['Article_total_len'], 2)
df_temp = df_raw.groupby(['Meta_Publisher'],as_index=False).agg({'Article_sentiment':'mean'}).round(2)
df_temp.reset_index(drop=True,inplace=True)

In [ ]:
sentiment_flow = []
for ind,row in df_temp.iterrows():
      for index,word in enumerate(df_temp['Article_preprocess_txt'][ind]):
            if word in pos_words:
                  sentiment_flow.append(1)
            elif word in neg_words:
                  sentiment_flow.append(-1)
            else:
                  sentiment_flow.append(0)
          
      df_change_in_sentiment = pd.DataFrame()
      df_change_in_sentiment['sentiment'] = sentiment_flow
      fig = px.line(df_change_in_sentiment, x=df_change_in_sentiment.index, y="sentiment", title='Sentiment Flow')
      fig.show()

In [ ]:
def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

In [ ]:
df_raw = df_raw.reset_index(drop= True)

In [ ]:
for index,row in df_raw.iterrows():
    for col in ['Meta_Publisher', 'Meta_title','Meta_desc', 'Title','Article','Summary']:
        row[col] = row[col].strip().replace('\n','').replace('#','')
        row[col] = remove_punctuation(row[col])
        df_raw.iloc[index][col] = row[col]

In [ ]:
df_clean = df_raw[['Meta_Publisher', 'Published_Date_clean','Meta_title','Keywords','Authors','Authors_Count','Title','Meta_desc','Article','Summary']]

### Clustering

In [ ]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
corpus = df_clean['Article'].to_list()
corpus_embeddings = embedder.encode(corpus)
# Perform kmean clustering
num_clusters = 2
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_
df_clean['clusters'] = cluster_assignment

df_clusters_temp = pd.DataFrame(df_clean.groupby(['Meta_Publisher'] ,as_index=False)['clusters'].value_counts())
df_clusters_temp = pd.pivot_table(df_clusters_temp, values = 'count', index=['Meta_Publisher'], columns = 'clusters').reset_index().fillna(-1)
df_clusters_temp.columns = [str(col) for (col) in (df_clusters_temp.columns)]
df_clusters_temp['Dominant_Cluster'] = df_clusters_temp[['0','1']].astype('int').idxmax(axis=1)
df_clusters_temp.sort_values(['Dominant_Cluster']).head(100)
df_clusters_temp.Meta_Publisher.unique()

### Creating Features

### NLP Metrics

### Feature Analysis

### Final Thoughts

https://www.google.com/search?q=notepad%2B%2B&rlz=1C5CHFA_enSG938SG938&oq=no&aqs=chrome.2.69i57j0i433i512j0i512j0i433i512j0i131i433i512j69i61j69i60j69i61.3195j0j9&sourceid=chrome&ie=UTF-8
https://www.google.com/search?q=notepad%2B%2B&rlz=1C5CHFA_enSG938SG938&ei=sR4pZM2gDPWwseMPn6e68AU&start=10&sa=N&ved=2ahUKEwiNvc2Ex4r-AhV1WGwGHZ-TDl4Q8tMDegQIERAE&biw=1080&bih=1692&dpr=1

